In [1]:
import pandas as pd
covid_deaths = 3650 

In [2]:
url_populacao = 'http://api.sidra.ibge.gov.br/values/t/6579/P/2020/n6/all?formato=json'
municipios = pd.read_json(url_populacao)[1:]

In [3]:
municipios.head()

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N
1,6,Município,45,Pessoas,22728,2020,2020,1100015,Alta Floresta D'Oeste - RO,9324,População residente estimada
2,6,Município,45,Pessoas,109523,2020,2020,1100023,Ariquemes - RO,9324,População residente estimada
3,6,Município,45,Pessoas,5188,2020,2020,1100031,Cabixi - RO,9324,População residente estimada
4,6,Município,45,Pessoas,85893,2020,2020,1100049,Cacoal - RO,9324,População residente estimada
5,6,Município,45,Pessoas,16204,2020,2020,1100056,Cerejeiras - RO,9324,População residente estimada


In [4]:
municipios.shape

(5570, 11)

In [5]:
municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 1 to 5570
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   NC      5570 non-null   object
 1   NN      5570 non-null   object
 2   MC      5570 non-null   object
 3   MN      5570 non-null   object
 4   V       5570 non-null   object
 5   D1C     5570 non-null   object
 6   D1N     5570 non-null   object
 7   D2C     5570 non-null   object
 8   D2N     5570 non-null   object
 9   D3C     5570 non-null   object
 10  D3N     5570 non-null   object
dtypes: object(11)
memory usage: 478.8+ KB


In [6]:
municipios.rename(columns={"D2C": "Codigo", "D2N": "Municipio", "V":"Populacao"}, inplace=True)
municipios = municipios[['Codigo', 'Municipio', 'Populacao']]
municipios['Populacao'] = pd.to_numeric(municipios['Populacao'])

## 1) Inserindo Latitude e Longitude no dataset original

In [7]:
#Dataset com latitude e longitude dos municípios

lat_long = pd.read_csv('municipios.csv')

In [8]:
lat_long.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31
2,5200100,Abadiânia,-16.19700,-48.7057,0,52
3,3100203,Abaeté,-19.15510,-45.4444,0,31
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15


In [9]:
#transformando 'codigo' para inteiro

municipios['Codigo'] = municipios['Codigo'].astype('int')

In [10]:
#Fazendo o merge da tabela original com a latitude e longitude dos municípios

municipios = pd.merge(municipios, lat_long, how='left', left_on='Codigo', right_on='codigo_ibge')

In [11]:
municipios.head()

,Codigo,Municipio,Populacao,codigo_ibge,nome,latitude,longitude,capital,codigo_uf
0,1100015,Alta Floresta D'Oeste - RO,22728,1100015,Alta Floresta D'Oeste,-11.92830,-61.9953,0,11
1,1100023,Ariquemes - RO,109523,1100023,Ariquemes,-9.90571,-63.0325,0,11
2,1100031,Cabixi - RO,5188,1100031,Cabixi,-13.49450,-60.5520,0,11
3,1100049,Cacoal - RO,85893,1100049,Cacoal,-11.43430,-61.4562,0,11
4,1100056,Cerejeiras - RO,16204,1100056,Cerejeiras,-13.18700,-60.8168,0,11


In [12]:
#Excluindo colunas desnecessárias

municipios = municipios.drop(columns=['codigo_ibge','nome', 'capital','codigo_uf'])

In [13]:
municipios = pd.DataFrame(municipios)

In [14]:
municipios.head()

,Codigo,Municipio,Populacao,latitude,longitude
0,1100015,Alta Floresta D'Oeste - RO,22728,-11.92830,-61.9953
1,1100023,Ariquemes - RO,109523,-9.90571,-63.0325
2,1100031,Cabixi - RO,5188,-13.49450,-60.5520
3,1100049,Cacoal - RO,85893,-11.43430,-61.4562
4,1100056,Cerejeiras - RO,16204,-13.18700,-60.8168


In [15]:
municipios.shape

(5570, 5)

In [16]:
municipios.isnull().sum()

Codigo       0
Municipio    0
Populacao    0
latitude     0
longitude    0
dtype: int64

## 2) Utilizando o Folium

In [17]:
municipios_encontrados = municipios[municipios['Populacao'] <= covid_deaths]

In [21]:
municipios_encontrados.head()

,Codigo,Municipio,Populacao,latitude,longitude
32,1100908,Castanheiras - RO,2987,-11.42530,-61.9482
42,1101468,Pimenteiras do Oeste - RO,2148,-13.48230,-61.0471
43,1101476,Primavera de Rondônia - RO,2776,-11.82950,-61.3153
105,1302108,Japurá - AM,2251,-1.88237,-66.9291
167,1501253,Bannach - PA,3262,-7.34779,-50.3959


In [18]:
import folium

In [26]:
brasil = folium.Map(
    location=[-16.1237611, -59.9219642],    # Coordenadas Brasil 
    zoom_start=4
)

In [27]:
brasil

## 3) Marcando os municípios no mapa

In [28]:
for _, municipio in municipios_encontrados.iterrows():
    folium.Marker(
        location=[municipio['latitude'], municipio['longitude']],
        popup=municipio['Municipio'],
    ).add_to(brasil)

In [29]:
brasil

# Todo

- Retornar as mortes confirmadas (diárias) através de API
- Alterar Ícone dos marcadores
- Colorir marcador por estado (será preciso algum tratamento no df para isso? :D)